# Simulating the Arm

In [55]:
import numpy as np
from IPython.display import HTML, SVG, display
from matplotlib import pyplot as plt
from tqdm import tqdm
import sys
import pydot

from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder, LeafSystem
from pydrake.systems.planar_scenegraph_visualizer import ConnectPlanarSceneGraphVisualizer
from pydrake.systems.primitives import LogVectorOutput, WrapToSystem
from pydrake.all import StartMeshcat, MeshcatVisualizer, MeshcatVisualizerParams

## Arm in 2D

In [104]:
def arm_sim(time, initial_state, log=False, visualize=False, graph=False, meshcat=None,
            accuracy=1e-4, min_step=0, step_target=None, fixed_step=False):
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, 0.)
    parser = Parser(plant)
    parser.AddModels("./ball.urdf")
    plant.Finalize()

    n_pos = plant.num_positions()
    print(n_pos)

    if log:
        logger = LogVectorOutput(wrapto.get_output_port(0), builder)
    if meshcat:
        vis_params = MeshcatVisualizerParams(publish_period=0.01)
        MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat, params=vis_params)
    if visualize:
        T_VW = np.array([[1., 0., 0., 0.],
                         [0., 0., 1., 0.],
                         [0., 0., 0., 1.]])
        visualizer = ConnectPlanarSceneGraphVisualizer(
            builder, scene_graph, T_VW=T_VW, xlim=[-2, 2],
            ylim=[-2, 2], show=False)
        visualizer.start_recording()


    diagram = builder.Build()
    if graph:
        display(SVG(pydot.graph_from_dot_data(diagram.GetGraphvizString(max_depth=2))[0].create_svg()))
    context = diagram.CreateDefaultContext()
    simulator = Simulator(diagram, context)
    simulator.get_integrator().set_target_accuracy(accuracy)
    if step_target is not None:
        simulator.get_integrator().request_initial_step_size_target(step_target)
    simulator.get_integrator().set_requested_minimum_step_size(min_step)
    simulator.get_integrator().set_fixed_step_mode(fixed_step)
    simulator.Initialize()
    if meshcat:
        simulator.set_target_realtime_rate(1)
    context.SetContinuousState(initial_state)

    for t in tqdm(np.linspace(0, time, 100), file=sys.stdout):
        simulator.AdvanceTo(t)

    if log:
        log = logger.FindLog(context)
        time = log.sample_times()[1:]
        data = log.data()[:,1:]
    if visualize:
        visualizer.stop_recording()
        ani = HTML(visualizer.get_recording_as_animation().to_jshtml())
    if log and visualize:
        return time, data, ani
    if log:
        return time, data
    if visualize:
        return ani

In [103]:
arm_sim(1, np.array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]), visualize=True)

16


IndexError: Operand vector size 13 does not match this drake::systems::Supervector<double> size 31

## Ball Simulation in 3D

In [53]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [95]:
ball_sim(1, np.array([0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1]), meshcat=meshcat)

7
100%|████████████████████████████████████████| 100/100 [00:00<00:00, 101.23it/s]
